In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from pathlib import Path

#### Load data

In [4]:
df = pd.read_pickle('../../data/data_tables/df_exclusion.pkl')

In [5]:
df_l5 = df[df['assigned_layer'] == 'L5']

In [6]:
len(df_l5)

5858

In [7]:
df_l5.loc[:, 'exclude'] = False

/tmp/ipykernel_1918179/1159152542.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l5.loc[:, 'exclude'] = False


#### Exclude L5 neurons based on cluster membership

In [8]:
NUM_CLUSTERS = 15

In [9]:
# Load manual validation table for layer 5.
df_manual = pd.read_pickle(
    Path('../preprocessing/neuron_preprocessing/data/morphology_validation_l5.pkl')
)
df_manual['natural_termination'] = (df_manual['recons_type'] == 'NT').values

# Merge manual validation table with cluster assignment
df_merge = pd.merge(
    df_l5, df_manual[['segment_id', 'natural_termination']], on=['segment_id']
)

# Exclude clusters with more than 50% cells with reconstruction issues. TODO: check criterion or do manually?
# Only apply if at least 10 cells in cluster were validated.
excluded_clusters = []
for c in range(1, NUM_CLUSTERS + 1):
    subset = df_merge[df_merge.cluster_L5 == c]
    if len(subset) > 15:
        if subset.natural_termination.value_counts(normalize=True)[False] > 0.5:
            excluded_clusters.append(c)
print(f' Exclude L5 exclusion clusters: {excluded_clusters}.')

if len(excluded_clusters) > 0:
    # Map exclusion to all L5 cells in excluded clusters.
    df_l5.loc[df_l5['cluster_L5'].isin(excluded_clusters), 'exclude'] = True
    df_l5.loc[df_l5['cluster_L5'].isin(excluded_clusters), 'exclusion_criterion'] = (
        'exclude_cluster'
    )
    percent = df_l5.exclude.value_counts(normalize=True)[1] * 100
    print(f'L5 exclusion: {percent:.0f}%; counts:\n{df_l5.exclude.value_counts()}.')

 Exclude L5 exclusion clusters: [5, 7, 8, 11].
L5 exclusion: 21%; counts:
False    4600
True     1258
Name: exclude, dtype: int64.


In [10]:
df = pd.concat([df[df.assigned_layer != 'L5'], df_l5])

In [11]:
len(df)

41993

In [20]:
# df.to_pickle('../data/df_exclusion.pkl')